In [327]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

def okt_tokenizer(x):
    return okt.morphs(x)
def okt_tokenizer2(x):
    return okt.nouns(x)


from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Data

In [376]:
class tdf(object) :
    
    def __init__(self):
        self.data=pd.read_csv(str(input('review 버전을 입력해주세요')))
        
    def get_matrix(self,by,min_df=0.15):
        review = self.data
        if by == 'lecture':
            self.tf_data = review.groupby(['name','prof'])['review'].agg(lambda x : ''.join(x)).reset_index()
        elif by == 'prof':
            self.tf_data = review.groupby(['prof'])['review'].agg(lambda x : ''.join(x)).reset_index()
        self.by=by
        self.corpus = self.tf_data['review'].tolist()
        
        tok_corpus=[]
        for doc in self.corpus:
            tok_corpus.append(okt_tokenizer(doc))
        self.tok_corpus = tok_corpus
        
        self.tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, min_df=min_df).fit(self.corpus)
        self.tfidf_matrix = self.tfidf.transform(self.corpus).toarray()
        
        self.tfidf_tag = TfidfVectorizer(tokenizer=okt_tokenizer2, min_df=(min_df-0.05)).fit(corpus)
        self.tfidf_matrix_tag = self.tfidf_tag.transform(corpus).toarray()
        
        self.cos_mat=cosine_similarity(sparse.csr_matrix(self.tfidf_matrix))
        
    def get_similar(self):
        tf_data = self.tf_data
        cos_mat = self.cos_mat
        
        tfidf_tag = self.tfidf_tag
        tfidf_matrix_tag = self.tfidf_matrix_tag
        
        tok_corpus = self.tok_corpus
    
        by = self.by
        
        if by == 'lecture':
            name = input('강의 이름을 입력해 주세요')
            prof = input('교수 이름을 입력해 주세요')
            idx = tf_data.query('name == @name & prof == @prof').index[0]
            
        elif by =='prof' :
            prof = input('교수 이름을 입력해 주세요')
            idx = tf_data.query('prof == @prof').index[0]
            
        nouns = np.argsort(tfidf_matrix_tag[idx,:])[::-1][:5]
        
        hash_tags=[]
        for hash_tag in nouns :
            hash_tags.append(tfidf_tag.get_feature_names()[hash_tag])
        print(hash_tags)
        
        in_ = tok_corpus[idx]
        sim_idx = np.argsort(cos_mat[idx,:])[::-1][1:50]
        jac_sim={}

        for idxs in sim_idx:
            compute_ = tok_corpus[idxs]
            union = set(in_).union(set(compute_))
            intersection = set(in_).intersection(set(compute_))
            jac_sim[idxs] = len(intersection)/len(union)

        out_idx = (pd.Series(jac_sim).sort_values(ascending=False)[:5]).index
        display(tf_data.iloc[out_idx,:])

            

In [379]:
tf = tdf()
tf.get_matrix('prof')

review 버전을 입력해주세요review107.csv


In [390]:
tf.get_similar()

교수 이름을 입력해 주세요안성만
['수업', '질문', '책', '출석', '자체']


,prof,review
445,박미애,일단 수업때 출튀를 많이 했었습니다 그런데 정말 수업 열심히 듣고 나중에 열심히 공...
472,박용근,학생들이 편하게 수업에 임하고 시험을 볼 수 있도록 중요 핵심을 뽑아 정리해주심 ...
774,이국철,솔직히 이 교수님이랑 성향이 맞으면 굉장히 좋음 난 교수님이 수업과 관련없는 이야...
769,이경화,강의력은 손에 꼽을 정도로 잘하시고 수업 중에 딴 짓을 못해 다들 열심히 하는 분위...
938,이찬영,시험 전주에 시험문제 다 알려주세요 그러다보니 수업 잘 안듣는 분들도 계세요 그...


## 1013 연습장

In [304]:
review = pd.read_csv('review107.csv')

In [345]:
tf_data = review.groupby(['name','prof'])['review'].agg(lambda x : ''.join(x)).reset_index()
corpus = tf_data['review'].tolist()

tfidf = TfidfVectorizer(tokenizer=okt_tokenizer2, min_df=0.1).fit(corpus)
tfidf_matrix = tfidf.transform(corpus).toarray()

In [359]:
nouns = np.argsort(tfidf_matrix[3139,:])[::-1][:5]

In [360]:
for idx in nouns :
    print(tfidf.get_feature_names()[idx])

과제
팀
시험
수업
시간


In [391]:
import pickle

# save
#with open('tf_moudle_class_1013.pickle', 'wb') as f:
#    pickle.dump(tf, f, pickle.HIGHEST_PROTOCOL)

# load
with open('tf_moudle_class_1013.pickle', 'rb') as f:
    tf = pickle.load(f)

### 해야할 것 
1) 단어중에 몇개만 고르자  
2) 그걸로 시각화 하자 